In [1]:
pip install transformers bitsandbytes>=0.39.0 -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    device_map="auto",
    # load_in_4bit=True
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the disk and cpu.


In [86]:
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
device = "cuda" if torch.cuda.is_available() else "cpu"

In [88]:
model_inputs = tokenizer.apply_chat_template(
    [
        {"role": "system", "content": "You are a friendly chatbot. answer me only use korean. context의 내용만 보고 출처와 함께 대답해. 만약 context에 질문에 대한 답이 없다면 '주어진 정보로 대답할 수 없습니다.'라고만 말해"},
        {"role": "context", "content": context},
        {"role": "user", "content": question},
    ],
    return_tensors='pt',
    
).to(device)
output = tokenizer.decode(model_inputs[0], tokenize=True)
model_input = tokenizer([output], return_tensors='pt').to(device)

In [95]:
from transformers import TextStreamer

streamer = TextStreamer(tokenizer)

def on_finalized_text(text: str, stream_end: bool = False):
    print(text, flush=True, end="")
streamer.on_finalized_text = on_finalized_text

generated_ids = model.generate(
    **model_input, 
    streamer=streamer,
    pad_token_id=tokenizer.eos_token_id, 
    do_sample=True, 
    max_new_tokens=512
)

tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a friendly chatbot. answer me only use korean. context의 내용만 보고 출처와 함께 대답해. 만약 context에 질문에 대한 답이 없다면 '주어진 정보로 대답할 수 없습니다.'라고만 말해<|eot_id|><|start_header_id|>context<|end_header_id|>

다양한 종류: 감자의 종류는 수천 가지가 있으며, 각기 다른 색깔, 크기, 맛을 가지고 있습니다. 대표적인 종류로는 러셋 감자, 유콘 골드, 레드 포테이토 등이 있습니다.	출처:./문서/potato.txt
종류	출처:./문서/sweetpotato.txt
꽃: 감자 식물은 보통 흰색, 분홍색, 보라색, 또는 파란색 꽃을 피웁니다.
잎: 잎은 초록색이고, 잎자루를 가지고 있으며, 잎의 가장자리가 물결 모양입니다.
3. 재배 및 생산	출처:./문서/potato.txt
가족: 가지과 (Solanaceae)
형태: 감자는 지하에 있는 덩이줄기(혹은 괴경)가 식용 부분입니다. 이 덩이줄기는 영양분을 저장하는 역할을 합니다.	출처:./문서/potato.txt
주요 생산국: 중국, 인도, 러시아, 우크라이나, 미국 등이 감자의 주요 생산국입니다.
4. 영양 성분 및 건강 효과	출처:./문서/potato.txt
3. 재배 및 생산
재배 조건: 감자는 서늘한 기후를 좋아하며, 최적 성장 온도는 15~20도씨입니다. 배수가 잘 되는 토양이 필요합니다.	출처:./문서/potato.txt
1. 역사 및 기원
기원: 감자는 약 7,000~10,000년 전 페루와 볼리비아의 안데스 산맥 지역에서 처음 재배되었습니다.	출처:./문서/potato.txt
감자는 전 세계적으로 사랑받는 식품으로, 다양한 요리에

KeyboardInterrupt: 

In [43]:
context = '''다양한 종류: 감자의 종류는 수천 가지가 있으며, 각기 다른 색깔, 크기, 맛을 가지고 있습니다. 대표적인 종류로는 러셋 감자, 유콘 골드, 레드 포테이토 등이 있습니다.	출처: ./문서/potato.txt
종류	출처: ./문서/sweetpotato.txt
꽃: 감자 식물은 보통 흰색, 분홍색, 보라색, 또는 파란색 꽃을 피웁니다.
잎: 잎은 초록색이고, 잎자루를 가지고 있으며, 잎의 가장자리가 물결 모양입니다.
3. 재배 및 생산	출처: ./문서/potato.txt
가족: 가지과 (Solanaceae)
형태: 감자는 지하에 있는 덩이줄기(혹은 괴경)가 식용 부분입니다. 이 덩이줄기는 영양분을 저장하는 역할을 합니다.	출처: ./문서/potato.txt
주요 생산국: 중국, 인도, 러시아, 우크라이나, 미국 등이 감자의 주요 생산국입니다.
4. 영양 성분 및 건강 효과	출처: ./문서/potato.txt
3. 재배 및 생산
재배 조건: 감자는 서늘한 기후를 좋아하며, 최적 성장 온도는 15~20도씨입니다. 배수가 잘 되는 토양이 필요합니다.	출처: ./문서/potato.txt
1. 역사 및 기원
기원: 감자는 약 7,000~10,000년 전 페루와 볼리비아의 안데스 산맥 지역에서 처음 재배되었습니다.	출처: ./문서/potato.txt
감자는 전 세계적으로 사랑받는 식품으로, 다양한 요리에 활용될 수 있으며, 건강에도 좋은 영향을 미치는 중요한 식량 자원입니다.	출처: ./문서/potato.txt
건강 효과:
에너지 공급: 감자는 탄수화물이 풍부해 에너지 공급원으로 좋습니다.
소화 개선: 식이섬유가 풍부해 소화 건강에 도움이 됩니다.	출처: ./문서/potato.txt
5. 요리 및 활용
요리법: 감자는 다양하게 요리될 수 있습니다. 감자튀김, 매시드 포테이토, 감자 샐러드, 구운 감자 등 여러 가지 요리법이 있습니다.	출처: ./문서/potato.txt'''
question =  '감자의 종류'

In [94]:
AutoModelForCausalLM.from_pretrained

<bound method _BaseAutoModelClass.from_pretrained of <class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>>